# DATA 102 Final Project Code 

Group members: Bridget Nix, Danny Ticknor, Hannah Chea, Meer Wu

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Loading & Cleaning Data

## Valley Fever data

Dataset is from https://github.com/valleyfever/valleyfevercasedata?fbclid=IwAR1V_iaZ-LUItfFjXri8mV7qsVTl5KFh5F1NRb5-zn6C-EEF_-oI4lHerrM

In [46]:
#Upload dataset
valley_fever = pd.read_csv("valley_fever.csv")

In [47]:
#Turn dates into single column
valley_fever = valley_fever.melt(id_vars=['State', 'County','FIPS'], var_name='Date', value_name='Cases')
valley_fever['Date'] = pd.to_datetime(valley_fever['Date'], format='%Y/%m').dt.to_period('m')

#Adding state FIPS code
valley_fever.loc[(valley_fever.State == 'AZ'),'statefips']=4
valley_fever.loc[(valley_fever.State == 'CA'),'statefips']=6
valley_fever.loc[(valley_fever.State == 'NV'),'statefips']=32
valley_fever['statefips'] = valley_fever['statefips'].astype(int)
valley_fever['countyfips'] = valley_fever['FIPS'].astype(int)
valley_fever = valley_fever.drop(columns=['FIPS'])

In [48]:
valley_fever

,State,County,Date,Cases,statefips,countyfips
0,AZ,Apache,2000-01,1,4,4001
1,AZ,Cochise,2000-01,1,4,4003
2,AZ,Coconino,2000-01,1,4,4005
3,AZ,Gila,2000-01,1,4,4007
4,AZ,Graham,2000-01,0,4,4009
...,...,...,...,...,...,...
17275,NV,Nye,2015-12,0,32,32023
17276,NV,Pershing,2015-12,0,32,32027
17277,NV,Storey,2015-12,0,32,32029
17278,NV,Washoe,2015-12,2,32,32031


## Census Population data and combining with Valley Fever data

Census population data was from https://data.census.gov/cedsci/table?q=total&g=0400000US04.050000,06.050000,32.050000&tid=ACSDT1Y2019.B01003

In [49]:
#Upload dataset
population = pd.read_csv("population_2010_to_2014.csv")

In [50]:
#Dataset header was weird so we select every row under
population = population.iloc[1:, 1:3]
#Add columns for time and place
population.columns = ['Place', 'Size']
population['Place'] = population['Place'].str.split(', ')
population['State'] = population['Place'].str[1]
population['County'] = population['Place'].str[0].str.split().str[0]
population['Size'] = population['Size'].astype(int)
population = population.loc[:, ['State', 'County', 'Size']].replace({'Arizona':'AZ', 'California':'CA', 'Nevada':'NV'})

In [51]:
population

,State,County,Size
1,AZ,Apache,72142
2,AZ,Cochise,130807
3,AZ,Coconino,135817
4,AZ,Gila,53242
5,AZ,Graham,37311
...,...,...,...
86,NV,Pershing,6741
87,NV,Storey,3934
88,NV,Washoe,429985
89,NV,White,10043


In [52]:
#Merge population dataset with valley_fever data to get valley fever cases per 100,000
valley_fever_and_pop = valley_fever.merge(population, on=['State', 'County'])
valley_fever_and_pop['Cases per 100,000'] = valley_fever_and_pop['Cases'] * 100000 / valley_fever_and_pop['Size']
valley_fever_and_pop = valley_fever_and_pop[['State', 'County', 'Date', 'Cases', 'statefips', 'countyfips', 'Size', 'Cases per 100,000']]

In [53]:
valley_fever_and_pop

,State,County,Date,Cases,statefips,countyfips,Size,"Cases per 100,000"
0,AZ,Apache,2000-01,1,4,4001,72142,1.386155
1,AZ,Apache,2000-02,0,4,4001,72142,0.000000
2,AZ,Apache,2000-03,0,4,4001,72142,0.000000
3,AZ,Apache,2000-04,0,4,4001,72142,0.000000
4,AZ,Apache,2000-05,0,4,4001,72142,0.000000
...,...,...,...,...,...,...,...,...
13627,NV,Washoe,2015-08,0,32,32031,429985,0.000000
13628,NV,Washoe,2015-09,1,32,32031,429985,0.232566
13629,NV,Washoe,2015-10,1,32,32031,429985,0.232566
13630,NV,Washoe,2015-11,0,32,32031,429985,0.000000


## Ozone data

Our Ozone data is from https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-Ozone-Concentrations-2011/372p-dx3h/data was filtered with statefips == 4, 6, 32 on the website before downloading

In [54]:
#Read data
ozone = pd.read_csv("ozone.csv")

In [55]:
def clean_ozone(df):
    #Drop weird first column
    df = df.drop('Unnamed: 0', axis=1)
    
    #Convert date to datetime
    df['Date'] = pd.to_datetime(df['date']).dt.to_period('m')
    
    #Add state abbreviations
    df.loc[df['statefips']==4, 'State'] = 'AZ'
    df.loc[df['statefips']==6, 'State'] = 'CA'
    df.loc[df['statefips']==32, 'State'] = 'NV'
    
    #Select columns of interest
    df = df.merge(valley_fever[['County','countyfips']].groupby(['County','countyfips']).count().reset_index(), on='countyfips')
    df = df.loc[:, ['Date', 'statefips', 'countyfips', 'ds_o3_pred', 'State', 'County']].groupby(by=['Date', 'statefips', 'countyfips', 'State', 'County']).mean().reset_index()
    return df

ozone = clean_ozone(ozone)

In [56]:
ozone

,Date,statefips,countyfips,State,County,ds_o3_pred
0,2011-01,4,4001,AZ,Apache,41.927685
1,2011-01,4,4003,AZ,Cochise,44.131680
2,2011-01,4,4005,AZ,Coconino,43.223185
3,2011-01,4,4007,AZ,Gila,43.131209
4,2011-01,4,4009,AZ,Graham,43.493020
...,...,...,...,...,...,...
4315,2014-12,32,32027,NV,Pershing,30.025371
4316,2014-12,32,32029,NV,Storey,29.192694
4317,2014-12,32,32031,NV,Washoe,29.264034
4318,2014-12,32,32033,NV,White Pine,36.162299


## PM 2.5 data

Our Ozone data is from https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2011/fcqm-xrf4/data was filtered with statefips == 4, 6, 32 on the website before downloading.

In [57]:
#Read data
pm = pd.read_csv("pm.csv")

C:\Users\cheah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Added missing values that somehow got lost when downloading the data
pm.loc[pm.shape[0]-1, 'longitude'] = -118.25827
pm.loc[pm.shape[0]-1, 'ds_pm_pred'] = 13.9273
pm.loc[pm.shape[0]-1, 'ds_pm_stdd'] = 2.9059

def clean_pm(df):
    #Drop weird first column
    df = df.drop('Unnamed: 0', axis=1)
    
    #Convert date to datetime
    df['Date'] = pd.to_datetime(df['date']).dt.to_period('m')
    
    #Add state abbreviations
    df.loc[df['statefips']==4, 'State'] = 'AZ'
    df.loc[df['statefips']==6, 'State'] = 'CA'
    df.loc[df['statefips']==32, 'State'] = 'NV'
    
    #Select columns of interest
    df = df.merge(valley_fever[['County','countyfips']].groupby(['County','countyfips']).count().reset_index(), on='countyfips')
    df = df.loc[:, ['Date', 'statefips', 'countyfips', 'ds_pm_pred', 'State', 'County']].groupby(by=['Date', 'statefips', 'countyfips', 'State', 'County']).mean().reset_index()
    return df

pm = clean_pm(pm)

In [ ]:
pm

## Drought data

Drought data from https://droughtmonitor.unl.edu/Data/DataDownload/ComprehensiveStatistics.aspx

In [ ]:
#Read data
drought = pd.read_csv("drought.csv")

In [ ]:
#Select columns of interest and clarify columns
drought = drought.iloc[:, [0] + list(range(2,10))]
drought.columns = ['Date', 'County', 'State', 'None', 'D0', 'D1', 'D2', 'D3', 'D4']

#Convert date to datetime
drought['Date'] = pd.to_datetime(drought['Date'], format='%Y%m%d').dt.to_period('m')

#Remove the word "County" from county column
drought['County'] = drought['County'].str.split().str[0]

#Groupby date, county, state
drought = drought.groupby(by=['Date', 'County', 'State']).mean().reset_index().sort_values(by='County')

#Clarify names for columns
drought = drought.rename(columns={'None':'No drought', 'D0':'D0: Abnormally Dry', 'D1':'D1: Moderate Drought', 'D2':'D2: Severe Drought', 'D3':'D3: Extreme Drought', 'D4':'D4: Exceptional Drought'})

In [ ]:
drought

## Climate data: Temperature and Percipitation

Temperature and percipitation data from https://www.ncdc.noaa.gov/cag/county/mapping/4/tavg/202103/1/value.

In [ ]:
#Read datasets
ca_temps = pd.read_csv("ca_temps.csv")
az_temps = pd.read_csv("az_temps.csv")
nv_temps = pd.read_csv("nv_temps.csv")

ca_precipitation = pd.read_csv("ca_precipitation.csv")
az_precipitation = pd.read_csv("az_precipitation.csv")
nv_precipitation = pd.read_csv("nv_precipitation.csv")


In [ ]:
def clean_temps(df):
    #Select columns of interest and clarify columns
    df = df.iloc[3:, [0,1,2,3]]
    df.columns = ['State', 'County', 'Date', 'Avg Temp (F)']
    
    #Clean names in state and county columns
    df['State'] = df['State'].str[:2]
    df['County'] = df['County'].str.split().str[0]
    
    #Convert date to datetime
    df['Date'] = df['Date'].astype(str)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.to_period('m')
    df['Avg Temp (F)'] = df['Avg Temp (F)'].astype(float)
    return df

def clean_precipitation(df):
    #Select columns of interest & clarify columns
    df = df.iloc[3:, [0,1,2,3]]
    df.columns = ['State', 'County', 'Date', 'Precipitation (inches)']
    
    #Clean names in state and county columns
    df['State'] = df['State'].str[:2]
    df['County'] = df['County'].str.split().str[0]
    
    #Convert date to datetime
    df['Date'] = df['Date'].astype(str)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.to_period('m')
    df['Precipitation (inches)'] = df['Precipitation (inches)'].astype(float)
    return df

ca_temps = clean_temps(ca_temps)
az_temps = clean_temps(az_temps)
nv_temps = clean_temps(nv_temps)

ca_precipitation = clean_precipitation(ca_precipitation)
az_precipitation = clean_precipitation(az_precipitation)
nv_precipitation = clean_precipitation(nv_precipitation)

#Combine all temperature datasets
temps = ca_temps.append(az_temps.append(nv_temps))

#Combine all percipitation datasets
precipitation = ca_precipitation.append(az_precipitation.append(nv_precipitation))

#Merge temeprature and percipitation datasets on Date, State, and County columns
climate = temps.merge(precipitation.merge(drought, on=['Date', 'State', 'County']), on=['State', 'County', 'Date'])
climate

# Merging all the datasets into <code>final</code> data

In [ ]:
#Merge Ozone and PM data
pm_ozone = pm.merge(ozone, on=['Date', 'State', 'County', 'statefips', 'countyfips'])

#Merge Valley fever & population data with PM & Ozone
valley_pop_pm_ozone = valley_fever_and_pop.merge(pm_ozone, on=['State', 'County','Date','statefips','countyfips'])

#Merge that dataset with climate data to create final dataset
final = valley_pop_pm_ozone.merge(climate, on=['State', 'Date','County'])

#Create column for Year and Month
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month

In [ ]:
final

# EDA

## Cases vs. Ozone Concentration per state

In [ ]:
#Scatter plots
fig, (ax0, ax1) = plt.subplots(2, 2, figsize = (13,11)) 
ca = final[final['State'] == 'CA']
sns.regplot(ax=ax0[0], x='ds_o3_pred', y='Cases per 100,000', data=ca, scatter_kws={'color': 'b', 'alpha':0.4}, line_kws={'color': 'k'}, ci=None)
sns.set_style(style='white')
ax0[0].set_xlabel('Mean Ozone Concentration (ppb)', fontsize=13);
ax0[0].set_ylabel('Cases Per 100,000 People',fontsize=13);
ax0[0].set_title('Cases vs. Ozone Concentration in CA', fontsize=15)

az = final[final['State'] == 'AZ']
sns.regplot(ax=ax0[1], x='ds_o3_pred', y='Cases per 100,000', data=az, scatter_kws={'color': 'r', 'alpha':0.4}, line_kws={'color': 'k'}, ci=None)
sns.set_style(style='white')
ax0[1].set_xlabel('Mean Ozone Concentration (ppb)', fontsize=13);
ax0[1].set_ylabel('Cases Per 100,000 People',fontsize=13);
ax0[1].set_title('Cases vs. Ozone Concentration in AZ', fontsize=15)

nv = final[final['State'] == 'NV']
sns.regplot(ax=ax1[0], x='ds_o3_pred', y='Cases per 100,000', data=nv, scatter_kws={'color': 'g', 'alpha':0.4}, line_kws={'color': 'k'}, ci=None)
sns.set_style(style='white')
ax1[0].set_xlabel('Mean Ozone Concentration (ppb)', fontsize=13);
ax1[0].set_ylabel('Cases Per 100,000 People',fontsize=13);
ax1[0].set_title('Cases vs. Ozone Concentration in NV', fontsize=15)

## Bar plots for Averages Cases for Each Month for each state

In [ ]:
#bar plots
fig, (ax0, ax1) = plt.subplots(2, 2, figsize = (13,11)) 

cases_by_month_ca = ca.groupby(by='Month').mean().loc[:, ['Cases per 100,000']].reset_index()
ax0[0].bar(cases_by_month_ca['Month'], cases_by_month_ca['Cases per 100,000'], color='b')
ax0[0].set_xlabel('Month', fontsize=13)
ax0[0].set_ylabel('Cases Per 100,000 People', fontsize=13)
ax0[0].set_title("Average Cases for Each Month in CA", fontsize=15)

cases_by_month_az = az.groupby(by='Month').mean().loc[:, ['Cases per 100,000']].reset_index()
ax0[1].bar(cases_by_month_az['Month'], cases_by_month_az['Cases per 100,000'], color='r')
ax0[1].set_xlabel('Month', fontsize=13)
ax0[1].set_ylabel('Cases Per 100,000 People', fontsize=13)
ax0[1].set_title("Average Cases for Each Month in AZ", fontsize=15)

cases_by_month_nv = nv.groupby(by='Month').mean().loc[:, ['Cases per 100,000']].reset_index()
ax1[0].bar(cases_by_month_nv['Month'], cases_by_month_nv['Cases per 100,000'], color='g')
ax1[0].set_ylabel('Cases Per 100,000 People', fontsize=13)
ax1[0].set_title("Average Cases for Each Month in NV", fontsize=15)
ax1[0].set_xlabel('Month', fontsize=13)


# Line plots

In [ ]:
#line plots
colors={'AZ':'#C60404', 'CA':'g', 'NV':'b'}
fig, axes = plt.subplots(3, 2, figsize=(8,8))

ax_list = axes.ravel()
i=0
vf_time = valley_fever.groupby(['State', 'Date']).sum().reset_index().sort_values(['State', 'Date'])
vf_time = vf_time[vf_time['Date'].dt.year.isin([2011, 2012, 2013, 2014])]

for state in vf_time['State'].unique():
    ax = ax_list[i*2]
    state_vf = vf_time[(vf_time['State']==state)&(vf_time['Date'].dt.year==2014)]
    ax.plot(range(1,len(state_vf)+1), state_vf['Cases'], label=state, color=colors[state])
    ax.set_xticks(range(1,len(state_vf)+1))
    ax.set_title(state+' {} (lowest)'.format(state_vf['Date'].dt.year.values[0]))
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of Cases')
    i+=1
i=0
for state in vf_time['State'].unique():
    ax = ax_list[i*2+1]
    if state == 'NV':
        state_vf = vf_time[(vf_time['State']==state)&(vf_time['Date'].dt.year==2012)]
    else:
        state_vf = vf_time[(vf_time['State']==state)&(vf_time['Date'].dt.year==2011)]
    ax.plot(range(1,len(state_vf)+1), state_vf['Cases'], label=state, color=colors[state])
    ax.set_xticks(range(1,len(state_vf)+1))
    ax.set_title(state+' {} (highest)'.format(state_vf['Date'].dt.year.values[0]))
    ax.set_xlabel('Month')
    ax.set_ylabel('Number of Cases')
    i+=1
plt.tight_layout()
plt.suptitle('Number of Cases over Time, by State', y=1.02, fontsize=13);




## Heatmap on <code>final</code> data

In [ ]:
features = ['Cases per 100,000', 'ds_pm_pred', 'ds_o3_pred', 'Avg Temp (F)', 'Precipitation (inches)', 'No drought', 'D0: Abnormally Dry', 'D1: Moderate Drought', 'D2: Severe Drought', 'D3: Extreme Drought', 'D4: Exceptional Drought']
X = final.loc[:, features]

In [ ]:
plt.figure(figsize = (16,16))
heatmap = sns.heatmap(data=X.corr(), annot=True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', fmt='.2g')

## Pairplot on <code>final</code> data

In [ ]:
plt.figure(figsize = (16,16))
pairplot = sns.pairplot(data=X, kind='scatter', diag_kind='kde')

# Q1: Causal Inference: Instrumental Variable & 2-stage OLS

Code adapted from Lab 7 from the 102 course.

In [ ]:
pip install statsmodels

In [ ]:
import statsmodels.api as sm

causal = pd.DataFrame(final, copy=True)
causal = causal[causal['Year']==2014]

def fit_OLS_model(data, outcome_var, treatment_var):
    """
    Fits an OLS model from data.
    
    Inputs:
        df: pandas DataFrame
        treatment_var: name of variable we regress on
        outcome_var: name of variable we are regressing
    Outputs:
        fitted_model: model with OLS regression results
    """
    outcome = data[outcome_var]
    treatment = data[treatment_var]
    #we include an intercept term for every regression
    treatment = sm.add_constant(treatment)
    model = sm.OLS(outcome, treatment, intercept=True).fit()
    return(model)

# Fit OLS parameters to predict ozone from precipitation
temp_coeff_model = fit_OLS_model(causal, 'ds_o3_pred', 'Precipitation (inches)')
print(temp_coeff_model.summary())

# Compute predictions for ozone (stage 1)
intercept = temp_coeff_model.params[0]
temp_coeff = temp_coeff_model.params[1]
ozone_predictions = intercept + temp_coeff*causal['Precipitation (inches)']

# Add the predictions to the student_data dataframe
causal['pred_ozone'] = ozone_predictions


# Fit OLS parameters to predict cases from the predicted ozone
ozone_coeff_model = fit_OLS_model(causal, 'Cases per 100,000', "pred_ozone")
ozone_coeff = ozone_coeff_model.params[1]
print(ozone_coeff_model.summary())
print('Estimated ATE using 2SLS: {}'.format(ozone_coeff))

# Q2: GLM vs. Nonparametric method

## GLM for Bootstrap 

Code based off of Lecture 12 on 102 website

In [ ]:
pip install statsmodels

In [ ]:
pip install pymc3

In [ ]:
pip install arviz

In [ ]:
from sklearn.model_selection import train_test_split

#split final dataset into test and training
train, test = train_test_split(final, test_size = .3)

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns


import pymc3 as pm
from pymc3 import glm
import arviz as az

%matplotlib inline

import numpy.random as rnd
import matplotlib.pyplot as plots
import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.metrics import mean_squared_error


plt.style.use("fivethirtyeight")


#bootstrapped GLM attempt 1

def bootstrap_xy(X, y, fnc, w=None, B=1000, plot=True):
    d = X.shape[1]
    N = X.shape[0]
    w_hat = fnc(X, y)
    w_boot = np.zeros(shape=(B,d))
    for b in range(B):
        bootstrap_indices = rnd.choice(np.arange(N), N)
        bootstrap_X = X.iloc[bootstrap_indices, :]
        bootstrap_y = y.iloc[bootstrap_indices]
        w_boot[b,:] = fnc(bootstrap_X, bootstrap_y)
    if plot:
        plt.scatter(w_boot[:,0], w_boot[:,1], c='b')
        plt.scatter(w_hat[0], w_hat[1], c='r', marker='x', s=300)
        if w:
            plt.scatter(w[0], w[1], c='g', marker='x', s=300)
        plt.show()
    return w_boot

#Example showing code used for all three: Gaussian, Poisson, and NegBin
#Will only be incorporating NegBin because its errors most closely matched those of the #bootstrap

def lin_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.Gaussian()
    )
    results = model.fit()
    params = results.params
    
    return params

def poisson_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.Poisson()
    )
    results = model.fit()
    params = results.params
    
    return params

def negbin_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.NegativeBinomial()
    )
    results = model.fit()
    params = results.params
    
    return params

train
#In all 3 bootstrap, I first compared a Gaussian, a Poisson, and a Negative Binomial model fit #onto the data. The bootstrap errors most closely resembled the negative binomial model’s #errors, so I chose to use the negative binomial on all three. 

#We chose to incorporate all three states and all of the years in our bootstrap. This was to #ensure that the random forest could be properly compared to the GLM bootstrap. From here, #we isolated three key features, those features being rainfall, ozone, and temperature.


ok_filter = (train['Precipitation (inches)'] < 1)
ok_cases = train[ok_filter].sort_values('Date')
ok_cases["totals"] = np.cumsum(ok_cases['Cases'])
# Log-transform the counts, too
ok_cases["log_totals"] = np.log(ok_cases["totals"])
ok_cases

#We chose precipitation at lower than 1 inch daily #because this 
#also likely affects our variable of interest.



w_negbin_boot = bootstrap_xy(sm.add_constant(ok_cases['Precipitation (inches)']), ok_cases.totals, negbin_model)


beta_0, beta_1 = w_negbin_boot.std(axis = 0)
print(f"Bootstrap std error for const: {beta_0:.3f}")
print(f"Bootstrap std error for year: {beta_1:.3f}")

negbin_model = sm.GLM(
    ok_cases.totals, sm.add_constant(ok_cases['Precipitation (inches)']),
    family=sm.families.NegativeBinomial()
)
negbin_results1 = negbin_model.fit()
print(negbin_results1.summary())




In [ ]:
#bootstrapped GLM attempt 2

def bootstrap_xy(X, y, fnc, w=None, B=1000, plot=True):
    d = X.shape[1]
    N = X.shape[0]
    w_hat = fnc(X, y)
    w_boot = np.zeros(shape=(B,d))
    for b in range(B):
        bootstrap_indices = rnd.choice(np.arange(N), N)
        bootstrap_X = X.iloc[bootstrap_indices, :]
        bootstrap_y = y.iloc[bootstrap_indices]
        w_boot[b,:] = fnc(bootstrap_X, bootstrap_y)
    if plot:
        plt.scatter(w_boot[:,0], w_boot[:,1], c='b')
        plt.scatter(w_hat[0], w_hat[1], c='r', marker='x', s=300)
        if w:
            plt.scatter(w[0], w[1], c='g', marker='x', s=300)
        plt.show()
    return w_boot

def lin_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.Gaussian()
    )
    results = model.fit()
    params = results.params
    
    return params

def poisson_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.Poisson()
    )
    results = model.fit()
    params = results.params
    
    return params

def negbin_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.NegativeBinomial()
    )
    results = model.fit()
    params = results.params
    
    return params

ok_filter_2 = (train['Avg Temp (F)'] > 85)
ok_cases_2 = train[ok_filter_2].sort_values('Date')
ok_cases_2["totals"] = np.cumsum(ok_cases_2['Cases'])
# Log-transform the counts, too
ok_cases_2["log_totals"] = np.log(ok_cases_2["totals"])
ok_cases_2

#We chose temperature to be higher than 85 because the disease tends to be contracted in the #hotter, drier times of the day.

w_negbin_boot = bootstrap_xy(sm.add_constant(ok_cases_2['Avg Temp (F)']), ok_cases_2.totals, negbin_model)

beta_0, beta_1 = w_negbin_boot.std(axis = 0)
print(f"Bootstrap std error for const: {beta_0:.3f}")
print(f"Bootstrap std error for year: {beta_1:.3f}")

negbin_model = sm.GLM(
    ok_cases_2.totals, sm.add_constant(ok_cases_2['Avg Temp (F)']),
    family=sm.families.NegativeBinomial()
)
negbin_results2 = negbin_model.fit()
print(negbin_results2.summary())



In [ ]:
#bootstrapped GLM attempt 3

def bootstrap_xy(X, y, fnc, w=None, B=1000, plot=True):
    d = X.shape[1]
    N = X.shape[0]
    w_hat = fnc(X, y)
    w_boot = np.zeros(shape=(B,d))
    for b in range(B):
        bootstrap_indices = rnd.choice(np.arange(N), N)
        bootstrap_X = X.iloc[bootstrap_indices, :]
        bootstrap_y = y.iloc[bootstrap_indices]
        w_boot[b,:] = fnc(bootstrap_X, bootstrap_y)
    if plot:
        plt.scatter(w_boot[:,0], w_boot[:,1], c='b')
        plt.scatter(w_hat[0], w_hat[1], c='r', marker='x', s=300)
        if w:
            plt.scatter(w[0], w[1], c='g', marker='x', s=300)
        plt.show()
    return w_boot

def lin_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.Gaussian()
    )
    results = model.fit()
    params = results.params
    
    return params

def poisson_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.Poisson()
    )
    results = model.fit()
    params = results.params
    
    return params

def negbin_model(x, y): 
    model = sm.GLM(
        y, x,
        family=sm.families.NegativeBinomial()
    )
    results = model.fit()
    params = results.params
    
    return params

ok_filter_3 =(train['ds_o3_pred'] > 50)
ok_cases_3 = train[ok_filter_3].sort_values('Date')
ok_cases_3["totals"] = np.cumsum(ok_cases_3['Cases'])
# Log-transform the counts, too
ok_cases_3["log_totals"] = np.log(ok_cases_3["totals"])
ok_cases_3

#We chose ozone to be higher than 50ppm because this is #considered an unhealthy living #standard of air quality.


w_negbin_boot = bootstrap_xy(sm.add_constant(ok_cases_3['ds_o3_pred']), ok_cases_3.log_totals, negbin_model)

beta_0, beta_1 = w_negbin_boot.std(axis = 0)
print(f"Bootstrap std error for const: {beta_0:.3f}")
print(f"Bootstrap std error for year: {beta_1:.3f}")

negbin_model = sm.GLM(
    ok_cases_3.totals, sm.add_constant(ok_cases_3['ds_o3_pred']),
    family=sm.families.NegativeBinomial()
)
negbin_results3 = negbin_model.fit()
print(negbin_results3.summary())



## Random Forest

Code adapted from Lab 9 from 102 website

In [ ]:
X = train[['ds_pm_pred', 'ds_o3_pred', 'Avg Temp (F)', 'Precipitation (inches)']]
y = train['Cases per 100,000']

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#Fit Random Forest 
forest_model = RandomForestRegressor(max_features=1)
forest_model.fit(X, y)

train["forest_pred"] = forest_model.predict(X)
test["forest_pred"] = forest_model.predict(test[['ds_pm_pred','ds_o3_pred', 'Avg Temp (F)', 'Precipitation (inches)']])

In [ ]:
forest_model.feature_importances_

In [ ]:
#Calculate RMSE of training and test data
train_rmse = np.mean((train["forest_pred"] - train['Cases per 100,000']) ** 2) ** 0.5
test_rmse = np.mean((test["forest_pred"] - test['Cases per 100,000']) ** 2) ** 0.5
print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")

In [ ]:
#Predictions vs. True Values plot
fig, ax = plt.subplots(figsize=(8,6))
plt.scatter(test["Cases per 100,000"], test['forest_pred'])
ax.plot([0, 1], [0, 1], transform=ax.transAxes, c="red")
ax.set_ylabel('Predictions',fontsize=15);
ax.set_xlabel('True Values',fontsize=15);

# Comparing GLM Negative Binomial with Random Forest

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
print(f"Mean squared log error for Negative Binomial model (Precipitation) on test data: {mean_squared_log_error(test['Cases per 100,000'], negbin_results1.predict(sm.add_constant(test['Precipitation (inches)'])))}")
print(f"Mean squared log error for Negative Binomial model (Temperature) on test data: {mean_squared_log_error(test['Cases per 100,000'], negbin_results2.predict(sm.add_constant(test['Avg Temp (F)'])))}")
print(f"Mean squared log error for Negative Binomial model (Ozone) on test data: {mean_squared_log_error(test['Cases per 100,000'], negbin_results3.predict(sm.add_constant(test['ds_o3_pred'])))}")
print(f"Mean squared log error for Random Forest on test data: {mean_squared_log_error(test['Cases per 100,000'], test['forest_pred'])}")